## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [3]:
# Get imports
import pickle
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib qt

## And so on and so forth...

In [ ]:
# import helper functions
import camera_calibrator
import lane_smoothing
import binary_gradient
import perspective_transform
import measure_curvature
import original_perspective
import warp_lines

In [ ]:
# Files to undistort
calibration1 = ('./camera_cal/calibration1.jpg')
straight_lines1 = ('./test_images/straight_lines1.jpg')
straight_lines2 = ('./test_images/straight_lines2.jpg')
test1 = ('./test_images/test1.jpg')
test2 = ('./test_images/test2.jpg')
test3 = ('./test_images/test3.jpg')
test4 = ('./test_images/test4.jpg')
test5 = ('./test_images/test5.jpg')
test6 = ('./test_images/test6.jpg')

In [ ]:
def pipeline(image):
    
    # Directory of images for calibration 
    image_directory = glob.glob('camera_cal/calibration*.jpg')
      
    # Get matrix from calibration
    mtx, dist = camera_calibrator.calibration(image_directory)
    
    # smooth the lanes 
    smoothened_image = lane_smoothing.apply_clahe(image)
    
    # Undistored image 
    undistored_image = cv2.undistort(smoothened_image, mtx, dist, None, mtx)
    
    # set thresholds for edge detection 
    sobel_threshold = (170, 255)
    sobelx_threshold = (20, 100)
    
    # Binary Image 
    color_binary, binary_image = binary_gradient.binary_image(undistored_image, sobel_threshold, sobelx_threshold)
    
    # set top and bottom margins
    top_margin = 93
    bottom_margin = 450
    
    # Perspective Transform
    perspective_transformed_image = perspective_transform.get_transformed_perspective(binary_image, top_margin, bottom_margin)
    
    # set margins, sliding windows, pixels to recenter window
    margin = 100
    nwindows = 9
    minpixels = 50

    # convert back to original image space 
    color_warp = measure_curvature.get_colored_area(perspective_transformed_image, margin, nwindows, minpixels)
    warp_image = original_perspective.get_original_perspective(color_warp, top_margin, bottom_margin)

    result = cv2.addWeighted(smoothened_image, 1, warp_image, 0.3, 0)
    
    return result

In [ ]:
img = lambda fname: mpimg.imread(fname)
result = pipeline(img(test4))
plt.imshow(result)